In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
train.shape

> 데이터의 수 자체가 적음, 전처리가 중요

In [ ]:
train.isna().sum()

> 총 4개의 column에서 결측치 발생

In [ ]:
test.isna().sum()

> 총 4개의 column에서 결측치발생 

> train에서는 결측치가 발생하지않은 column : `자격유형`

In [ ]:
train_nunique = train.nunique()
train_nunique

In [ ]:
train_type_nunique = pd.DataFrame({
    'Data Type': train.dtypes,
    'Number of Unique Values': train.nunique()
})

train_type_nunique

> object이면서 nunique가 많은 값 : `공급유형`, `자격유형`, `지역`

In [ ]:
# 지역 data 합치기

train.loc[train.지역.isin(['경기도','서울특별시']), '지역'] = '수도권(서울/경기)'
train.loc[train.지역.isin(['경상남도','울산광역시','부산광역시','대구광역시','경상북도']), '지역'] = '경상도'
train.loc[train.지역.isin(['전라남도','광주광역시','전라북도']), '지역'] = '전라도'
train.loc[train.지역.isin(['충청남도','충청북도','대전광역시','세종특별자치시']), '지역'] = '충청도'

test.loc[test.지역.isin(['경기도','서울특별시']), '지역'] = '수도권(서울/경기)'
test.loc[test.지역.isin(['경상남도','울산광역시','부산광역시','대구광역시','경상북도']), '지역'] = '경상도'
test.loc[test.지역.isin(['전라남도','광주광역시','전라북도']), '지역'] = '전라도'
test.loc[test.지역.isin(['충청남도','충청북도','대전광역시','세종특별자치시']), '지역'] = '충청도'

In [ ]:
train['공급유형'].unique()

In [ ]:
# train, test column rename : column명이 긴 것을 간결화
train.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)': '지하철수',
                      '도보 10분거리 내 버스정류장 수': '버스수'}, inplace=True)
test.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)': '지하철수',
                     '도보 10분거리 내 버스정류장 수': '버스수'}, inplace=True)


# 결측치 통일 : '임대료','임대보증금' 컬럼에서 '-' 값을 NaN으로 변경

for col in ['임대료', '임대보증금']:
    train[col] = train[col].replace('-', np.nan).astype(float)
    test[col] = test[col].replace('-', np.nan).astype(float)

# 해당 컬럼을 float64 타입으로 변환
train['임대료'] = train['임대료'].astype(float)
train['임대보증금'] = train['임대보증금'].astype(float)

# 해당 컬럼을 float64 타입으로 변환
test['임대료'] = test['임대료'].astype(float)
test['임대보증금'] = test['임대보증금'].astype(float)

# 버스수, 지하철수의 결측치를 0으로 채움
# <위의 근거를 써주세요!>

train['버스수'].fillna(0, inplace=True)
train['지하철수'].fillna(0, inplace=True)

#C2411 자격유형 → 'A'
# <위의 근거를 써주세요!>
test[test['단지코드']=='C2411']['자격유형'].fillna('A')

#C2253 자격유형 → 'C'
# <위의 근거를 써주세요!>
test[test['단지코드']=='C2253']['자격유형'].fillna('C')

train.head()

In [ ]:
# Filter rows where '임대료' or '임대보증금' is NaN
missing_data = train[train['임대료'].isna() | train['임대보증금'].isna()]

# Display statistical summary and the first few rows of the missing data
summary = missing_data.describe(include='all')
summary


> `임대건물구분`에서 대부분의 행이 '상가'로 분류되어 있습니다.

> `공급유형`은 대부분 '임대상가'입니다.

> `자격유형`은 모두 'D'입니다.


In [ ]:
print("임대건물구분이 상가인 것의 임대료",train[train['임대건물구분'] == '상가']['임대료'].unique())
print("임대건물구분이 상가인 것의 임대보증금",train[train['임대건물구분'] == '상가']['임대보증금'].unique())
print("공급유형이 임대상가인 것의 임대료",train[train['공급유형']=='임대상가']['임대료'].unique())
print("공급유형이 임대상가인 것의 임대보증금",train[train['공급유형']=='임대상가']['임대보증금'].unique())
print("자격유형이 D인 것의 임대료",train[train['자격유형'] == 'D']['임대료'].unique())
print("자격유형이 D인 것의 임대보증금",train[train['자격유형'] == 'D']['임대보증금'].unique())

> 해당하는 조건을 해봤으나 모두 nan값이라 유의미한 결과를 도출할 수 없음

> 상가가 아닌 것의 데이터를 살펴보았습니다.

In [ ]:
train[(train['임대료'].isna()) & (train['임대건물구분'] != '상가')]

In [ ]:
test[(test['임대료'].isna()) & (test['임대건물구분'] != '상가')]

> 장기전세주택이란 월 임대료를 지불하지 않고 전세계약 방식으로 공급되는 공공임대주택을 말합니다. [출처](https://www.mylawstory.com/5606/)

>그러므로 장기전세의 임대료는 0으로 바꿔주는 것이 맞습니다


In [ ]:
#### 장기전세는 월 임대료를 지불하지않으므로 임대료 = 0 적용
train.loc[train['공급유형'] == '장기전세', '임대료'] = 0
test.loc[test['공급유형'] == '장기전세', '임대료'] = 0

> 전용면적과 임대료, 임대보증금이 높은 상관관계를 가지므로

> 공급유형이 상가가 아닌것들 중에 임대료와 임대보증금이 결측치인경우 지역별로 나눈 후 전용면적에 따른 회귀모델을 통해 임대료와 임대보증금의 결측치를 채워줍니다.

In [ ]:
#train data 상가아닌 것 임대료, 임대보증금 회귀예측

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Filter out rows where '공급유형' is '상가'
train_non_store = train[train['공급유형'] != '상가']

# List of unique regions
regions = train_non_store['지역'].unique()

# For storing predicted values
predicted_rent = []
predicted_deposit = []

# Iterate over each region
for region in regions:
    region_data = train_non_store[train_non_store['지역'] == region].copy()
    
    # Train a regression model for '임대료'
    X = region_data[~region_data['임대료'].isna()][['전용면적']]
    y = region_data[~region_data['임대료'].isna()]['임대료']
    if len(X) > 0:
        model_rent = LinearRegression()
        model_rent.fit(X, y)
        # Predict for missing values
        X_missing_rent = region_data[region_data['임대료'].isna()][['전용면적']]
        if len(X_missing_rent) > 0:
            pred_rent = model_rent.predict(X_missing_rent)
            predicted_rent.extend(pred_rent)
    
    # Train a regression model for '임대보증금'
    X = region_data[~region_data['임대보증금'].isna()][['전용면적']]
    y = region_data[~region_data['임대보증금'].isna()]['임대보증금']
    if len(X) > 0:
        model_deposit = LinearRegression()
        model_deposit.fit(X, y)
        # Predict for missing values
        X_missing_deposit = region_data[region_data['임대보증금'].isna()][['전용면적']]
        if len(X_missing_deposit) > 0:
            pred_deposit = model_deposit.predict(X_missing_deposit)
            predicted_deposit.extend(pred_deposit)

# Fill in the missing values in the original dataframe
missing_rent_indices = train_non_store[train_non_store['임대료'].isna()].index
for idx, value in zip(missing_rent_indices, predicted_rent):
    train.at[idx, '임대료'] = value

missing_deposit_indices = train_non_store[train_non_store['임대보증금'].isna()].index
for idx, value in zip(missing_deposit_indices, predicted_deposit):
    train.at[idx, '임대보증금'] = value

# Verify if the missing values are filled
missing_values = train[train['공급유형'] != '상가'][['임대료', '임대보증금']].isnull().sum()

missing_values

In [ ]:
# Filter out rows where '공급유형' is '상가'
test_non_store = test[test['공급유형'] != '상가']

# List of unique regions
regions = test_non_store['지역'].unique()

# For storing predicted values
predicted_rent = []
predicted_deposit = []

# Iterate over each region
for region in regions:
    region_data = test_non_store[test_non_store['지역'] == region].copy()
    
    # Train a regression model for '임대료'
    X = region_data[~region_data['임대료'].isna()][['전용면적']]
    y = region_data[~region_data['임대료'].isna()]['임대료']
    if len(X) > 0:
        model_rent = LinearRegression()
        model_rent.fit(X, y)
        # Predict for missing values
        X_missing_rent = region_data[region_data['임대료'].isna()][['전용면적']]
        if len(X_missing_rent) > 0:
            pred_rent = model_rent.predict(X_missing_rent)
            predicted_rent.extend(pred_rent)
    
    # Train a regression model for '임대보증금'
    X = region_data[~region_data['임대보증금'].isna()][['전용면적']]
    y = region_data[~region_data['임대보증금'].isna()]['임대보증금']
    if len(X) > 0:
        model_deposit = LinearRegression()
        model_deposit.fit(X, y)
        # Predict for missing values
        X_missing_deposit = region_data[region_data['임대보증금'].isna()][['전용면적']]
        if len(X_missing_deposit) > 0:
            pred_deposit = model_deposit.predict(X_missing_deposit)
            predicted_deposit.extend(pred_deposit)

# Fill in the missing values in the original dataframe
missing_rent_indices = test_non_store[test_non_store['임대료'].isna()].index
for idx, value in zip(missing_rent_indices, predicted_rent):
    test.at[idx, '임대료'] = value

missing_deposit_indices = test_non_store[test_non_store['임대보증금'].isna()].index
for idx, value in zip(missing_deposit_indices, predicted_deposit):
    test.at[idx, '임대보증금'] = value

# Verify if the missing values are filled
missing_values = test[test['공급유형'] != '상가'][['임대료', '임대보증금']].isnull().sum()

missing_values

> data type이 object인 것을 모두 인코딩(원-핫 인코딩 사용)

In [ ]:
# One-hot encode the object columns (except '단지코드')

# Identify the object columns to be encoded
object_columns = train.select_dtypes(include=['object']).columns.tolist()
object_columns.remove('단지코드')  # Exclude '단지코드'

# One-hot encode
train_encoded = pd.get_dummies(train, columns=object_columns)
test_encoded = pd.get_dummies(test, columns=object_columns)

# Display the first few rows of the encoded data
train_encoded.head()
train_encoded.columns

In [ ]:
train_store_data = train_encoded[train_encoded['임대건물구분_상가'] == 1].drop(columns=['임대료', '임대보증금'])
train_non_store_data = train_encoded[train_encoded['임대건물구분_상가'] != 1]

test_store_data = test_encoded[test_encoded['임대건물구분_상가'] == 1].drop(columns=['임대료', '임대보증금'])
test_non_store_data = test_encoded[test_encoded['임대건물구분_상가'] != 1]

In [ ]:
train_encoded.dtypes

In [ ]:
# X와 y 분리
X = train_store_data.drop(['단지코드','등록차량수'], axis=1)  # 'target'은 예측하고자 하는 타겟 변수입니다.
y = train_store_data['등록차량수']

# 학습 데이터와 검증 데이터로 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
lasso = Lasso(alpha=1.0)
ridge = Ridge(alpha=1.0)
elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5)

lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)
elastic_net.fit(X_train, y_train)

# 예측
lasso_pred = lasso.predict(X_val)
ridge_pred = ridge.predict(X_val)
elastic_net_pred = elastic_net.predict(X_val)

from sklearn.metrics import mean_absolute_error

# 성능 평가 (MAE)
lasso_mae = mean_absolute_error(y_val, lasso_pred)
ridge_mae = mean_absolute_error(y_val, ridge_pred)
elastic_net_mae = mean_absolute_error(y_val, elastic_net_pred)

min_mae = min(lasso_mae, ridge_mae, elastic_net_mae)

if min_mae == lasso_mae:
    best_model = lasso
    model_name = "Lasso"
elif min_mae == ridge_mae:
    best_model = ridge
    model_name = "Ridge"
else:
    best_model = elastic_net
    model_name = "ElasticNet"

print(f"Best Model: {model_name} with MAE: {min_mae}")



In [ ]:
# X와 y 분리
X = train_non_store_data.drop(['단지코드','등록차량수'], axis=1)  # 'target'은 예측하고자 하는 타겟 변수입니다.
y = train_non_store_data['등록차량수']

# 학습 데이터와 검증 데이터로 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
lasso = Lasso(alpha=1.0)
ridge = Ridge(alpha=1.0)
elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5)

lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)
elastic_net.fit(X_train, y_train)

# 예측
lasso_pred = lasso.predict(X_val)
ridge_pred = ridge.predict(X_val)
elastic_net_pred = elastic_net.predict(X_val)

from sklearn.metrics import mean_absolute_error

# 성능 평가 (MAE)
lasso_mae = mean_absolute_error(y_val, lasso_pred)
ridge_mae = mean_absolute_error(y_val, ridge_pred)
elastic_net_mae = mean_absolute_error(y_val, elastic_net_pred)

min_mae = min(lasso_mae, ridge_mae, elastic_net_mae)

if min_mae == lasso_mae:
    best_model = lasso
    model_name = "Lasso"
elif min_mae == ridge_mae:
    best_model = ridge
    model_name = "Ridge"
else:
    best_model = elastic_net
    model_name = "ElasticNet"

print(f"Best Model: {model_name} with MAE: {min_mae}")

